# Processing of Downloaded Dataset ( TO DO AFTER CREATION OF FOLDER STRUCTURE AND DOWNLOADING OF DATASET ON ZINDI )

In [ ]:
from zipfile import ZipFile
from google.colab import drive
import pandas as pd
import os, shutil
from multiprocessing import Process

In [2]:
######################################### REPO_PATH : Must be updated ###############################################
### like that: REPO_PATH = "drive/My Drive/Folder/sub-folder/subfolder_of_sub-folder/.../TUTORIAL:SPOT-THE-MASK-CHALLENGE/"
REPO_PATH = "drive/My Drive/REPOSITORIES/ZINDI-CHALLENGES/TUTORIAL:SPOT-THE-MASK-CHALLENGE/"

if REPO_PATH not in str(os.getcwd()) :
    drive.mount('/content/drive')

    os.chdir(REPO_PATH)
    print(f"Drive mounted at {os.getcwd()}") 
else:
    print(f"Drive already mounted at {os.getcwd()}") 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
Drive mounted at /content/drive/My Drive/REPOSITORIES/ZINDI-CHALLENGES/TUTORIAL:SPOT-THE-MASK-CHALLENGE


In [3]:
PATH = \
{
    "" : "",
    "CSV" : os.path.join( "DATASET", "CSV",  ),
    "IMAGES" : os.path.join( "DATASET", "IMAGES",  ),
    "garbage" : os.path.join( "DATASET", "IMAGES", "images"  ),
    "train" : os.path.join( "DATASET", "IMAGES", "train" ),
    "test" : os.path.join( "DATASET", "IMAGES", "test" ),
    "garbage_df" : os.path.join( "DATASET", "CSV", 'train_labels.csv' ),
    "zip_imgs" : os.path.join( "DATASET", "ZIP", 'images.zip' )
}

In [4]:
def unzip():
    print(f"[INFO] Unzipping")
    with ZipFile(PATH['zip_imgs'] , 'r') as zipObj:
        # Extract all the contents of zip file in current directory
        zipObj.extractall(path=PATH['IMAGES'],)
    print(f"[INFO] Succesful unzip")

In [11]:
def dataset():

    unzip()

    total_imgs = len(os.listdir( PATH['garbage'] ))
    print(f"[INFO] {total_imgs} images in garbage folder 'images' ")

    garbage_df = pd.read_csv( PATH["garbage_df"] )
    print(f"[INFO] {garbage_df.shape[0]} images in df ")


    print(f"[INFO] Creating folders to arrange train images by label ")
    mask_folder = os.path.join(PATH['train'], "1-MASK")
    no_mask_folder = os.path.join(PATH['train'], "0-NO_MASK")

    os.makedirs(name=mask_folder , exist_ok=True)
    os.makedirs(name=no_mask_folder , exist_ok=True)

    no_mask_filenames = garbage_df[garbage_df.target == 0].image.tolist()
    mask_filenames = garbage_df[garbage_df.target == 1].image.tolist()

    # Parallel moving
    print(f"[INFO] Moving NO_MASK images ")
    _ = []
    for name in no_mask_filenames:
        try:
            p = Process( target=shutil.move, args=(os.path.join( PATH['garbage'],name ), no_mask_folder))
            p.start()
            _ += [p]
        except:
            pass
    
    print(f"[INFO] Moving MASK images ")
    for name in mask_filenames:
        try:
            p = Process( target=shutil.move, args=(os.path.join( PATH['garbage'],name ), mask_folder))
            p.start()
            _ += [p]
        except:
            pass
    [p.join() for p in _]

    print(f"[INFO] Rename garbage folder : 'images' --> 'test'")
    os.rename( PATH['garbage'], PATH['test'],  )

    check = total_imgs == (garbage_df.shape[0] + len(os.listdir(PATH['test'])))
    if check:
        print(f"[INFO] Succesful processing")
        print(f"[INFO] {len(os.listdir(mask_folder))} images in folder '{mask_folder}' ")
        print(f"[INFO] {len(os.listdir(no_mask_folder))} images in folder '{no_mask_folder}' ")
        print(f"[INFO] {len(os.listdir(PATH['test']))} images in folder '{PATH['test']}'.")
    else:
        shutil.rmtree(PATH['IMAGES'])
        os.makedirs(PATH['IMAGES'], exist_ok=True)

        print(f"[INFO] Processing failed, re-run this function please.")

#Process dataset ( unzip, arrange in folders )

In [12]:
dataset()

[INFO] Succesful unzip
[INFO] 1817 images in garbage folder 'images' 
[INFO] 1308 images in df 
[INFO] Creating folders to arrange train images by label 
[INFO] Moving NO_MASK images 
[INFO] Moving MASK images 
[INFO] Rename garbage folder : 'images' --> 'test'
[INFO] Succesful processing
[INFO] 644 images in folder 'DATASET/IMAGES/train/1-MASK' 
[INFO] 664 images in folder 'DATASET/IMAGES/train/0-NO_MASK' 
[INFO] 509 images in folder 'DATASET/IMAGES/test'.


In [10]:
#### shutil.rmtree(PATH['IMAGES'])
#### os.makedirs(PATH['IMAGES'], exist_ok=True)